In [9]:
import json

from eppy.modeleditor import IDF
from io import StringIO
from datetime import datetime
import itertools
import math

def generate_idf_from_geojson(geojson: dict, idf_path: str):
    idf = IDF(StringIO("Version,25.1;"))

    # Simulation metadata
    idf.newidfobject("TIMESTEP", Number_of_Timesteps_per_Hour=4)
    idf.newidfobject("SITE:LOCATION",
                     Name="Site Location",
                     Latitude=40.0,
                     Longitude=-75.0,
                     Time_Zone=-5.0,
                     Elevation=200.0)
    idf.newidfobject("SIMULATIONCONTROL",
                     Do_Zone_Sizing_Calculation="No",
                     Do_System_Sizing_Calculation="No",
                     Do_Plant_Sizing_Calculation="No",
                     Run_Simulation_for_Weather_File_Run_Periods="Yes",
                     Run_Simulation_for_Sizing_Periods="No")
    today = datetime.today()
    idf.newidfobject("RUNPERIOD",
                     Name="RunPeriod1",
                     Begin_Month=today.month,
                     Begin_Day_of_Month=today.day,
                     End_Month=today.month,
                     End_Day_of_Month=today.day,
                     Use_Weather_File_Holidays_and_Special_Days="Yes",
                     Use_Weather_File_Daylight_Saving_Period="Yes",
                     Apply_Weekend_Holiday_Rule="Yes",
                     Use_Weather_File_Rain_Indicators="Yes",
                     Use_Weather_File_Snow_Indicators="Yes")

    idf.newidfobject("BUILDING",
                     Name="GeneratedBuilding",
                     North_Axis=0.0,
                     Terrain="Suburbs",
                     Loads_Convergence_Tolerance_Value=0.04,
                     Temperature_Convergence_Tolerance_Value=0.4,
                     Solar_Distribution="FullExterior",
                     Maximum_Number_of_Warmup_Days=25,
                     Minimum_Number_of_Warmup_Days=6)

    idf.newidfobject("GLOBALGEOMETRYRULES",
                     Starting_Vertex_Position="UpperLeftCorner",
                     Vertex_Entry_Direction="CounterClockWise",
                     Coordinate_System="Relative")

    # Schedule type limits
    idf.newidfobject("SCHEDULETYPELIMITS",
                     Name="Fraction",
                     Lower_Limit_Value=0.0,
                     Upper_Limit_Value=1.0,
                     Numeric_Type="Continuous",
                     Unit_Type="Dimensionless")
    
    idf.newidfobject("SCHEDULETYPELIMITS",
                     Name="Temperature",
                     Unit_Type="Temperature")

    # AlwaysOn schedule
    idf.newidfobject("SCHEDULE:COMPACT",
                     Name="AlwaysOn",
                     Schedule_Type_Limits_Name="Fraction",
                     Field_1="Through: 12/31",
                     Field_2="For: AllDays",
                     Field_3="Until: 24:00, 1")

    # Heating and cooling setpoint schedules
    idf.newidfobject("SCHEDULE:COMPACT",
                     Name="HeatingSetpoint",
                     Schedule_Type_Limits_Name="Temperature",
                     Field_1="Through: 12/31",
                     Field_2="For: AllDays",
                     Field_3="Until: 24:00, 20")  # 20°C

    idf.newidfobject("SCHEDULE:COMPACT",
                     Name="CoolingSetpoint",
                     Schedule_Type_Limits_Name="Temperature",
                     Field_1="Through: 12/31",
                     Field_2="For: AllDays",
                     Field_3="Until: 24:00, 24")  # 24°C

    # Materials and constructions
    idf.newidfobject("MATERIAL", Name="Wall Material", Roughness="Rough",
                     Thickness=0.2, Conductivity=0.5, Density=1400,
                     Specific_Heat=1000, Thermal_Absorptance=0.9,
                     Solar_Absorptance=0.7, Visible_Absorptance=0.7)
    idf.newidfobject("MATERIAL", Name="Floor Material", Roughness="Rough",
                     Thickness=0.1, Conductivity=1.0, Density=2000,
                     Specific_Heat=1000, Thermal_Absorptance=0.9,
                     Solar_Absorptance=0.7, Visible_Absorptance=0.7)
    idf.newidfobject("MATERIAL", Name="Roof Material", Roughness="Rough",
                     Thickness=0.15, Conductivity=0.3, Density=800,
                     Specific_Heat=1200, Thermal_Absorptance=0.9,
                     Solar_Absorptance=0.7, Visible_Absorptance=0.7)

    idf.newidfobject("CONSTRUCTION", Name="Wall Construction", Outside_Layer="Wall Material")
    idf.newidfobject("CONSTRUCTION", Name="Floor Construction", Outside_Layer="Floor Material")
    idf.newidfobject("CONSTRUCTION", Name="Roof Construction", Outside_Layer="Roof Material")

    zone_defs = []

    for feature in geojson["features"]:
        props = feature["properties"]
        zone_name = props.get("name", "UnnamedZone").replace(" ", "_")
        area_sqft = props.get("area_sqft", 100.0)
        area_m2 = area_sqft * 0.092903
        length = width = math.sqrt(area_m2)
        height = props.get("height_ft", 10.0) * 0.3048
        conditioned = props.get("conditioned", True)

        x0, y0 = feature["geometry"]["coordinates"][0][0]

        zone_defs.append({
            "name": zone_name, "x0": x0, "y0": y0,
            "length": length, "width": width,
            "height": height, "conditioned": conditioned
        })

        idf.newidfobject("ZONE",
                         Name=zone_name,
                         X_Origin=x0,
                         Y_Origin=y0,
                         Z_Origin=0)

        # Surfaces
        surfaces = {
            "South Wall": [(0, 0, 0), (0, 0, height), (length, 0, height), (length, 0, 0)],
            "East Wall": [(length, 0, 0), (length, 0, height), (length, width, height), (length, width, 0)],
            "North Wall": [(length, width, 0), (length, width, height), (0, width, height), (0, width, 0)],
            "West Wall": [(0, width, 0), (0, width, height), (0, 0, height), (0, 0, 0)],
            "Floor": [(0, width, 0), (length, width, 0), (length, 0, 0), (0, 0, 0)],
            "Roof": [(0, 0, height), (length, 0, height), (length, width, height), (0, width, height)]
        }

        for surf_name, verts in surfaces.items():
            surface_type = "Wall" if "Wall" in surf_name else "Roof" if surf_name == "Roof" else "Floor"
            construction = f"{surface_type} Construction"
            sun = "NoSun" if surface_type == "Floor" else "SunExposed"
            wind = "NoWind" if surface_type == "Floor" else "WindExposed"
            outside = "Ground" if surface_type == "Floor" else "Outdoors"

            vertex_dict = {}
            for i, v in enumerate(verts):
                vertex_dict[f"Vertex_{i+1}_Xcoordinate"] = v[0] + x0
                vertex_dict[f"Vertex_{i+1}_Ycoordinate"] = v[1] + y0
                vertex_dict[f"Vertex_{i+1}_Zcoordinate"] = v[2]

            idf.newidfobject("BUILDINGSURFACE:DETAILED",
                             Name=f"{zone_name}_{surf_name}",
                             Surface_Type=surface_type,
                             Construction_Name=construction,
                             Zone_Name=zone_name,
                             Outside_Boundary_Condition=outside,
                             Sun_Exposure=sun,
                             Wind_Exposure=wind,
                             Number_of_Vertices=4,
                             **vertex_dict)

        # Thermostat for conditioned zones
        if conditioned:
            idf.newidfobject("HVACTEMPLATE:THERMOSTAT",
                             Name=f"{zone_name}_Thermostat",
                             Heating_Setpoint_Schedule_Name="HeatingSetpoint",
                             Cooling_Setpoint_Schedule_Name="CoolingSetpoint")

            # HVAC system
            idf.newidfobject("HVACTEMPLATE:ZONE:IDEALLOADSAIRSYSTEM",
                             Zone_Name=zone_name,
                             Template_Thermostat_Name=f"{zone_name}_Thermostat")

    # Outputs
    output_vars = [
        "Zone Air Temperature",
        "Zone Mean Air Temperature",
        "Zone Infiltration Sensible Heat Gain",
        "Zone Infiltration Air Change Rate",
        "Zone Ideal Loads Supply Air Total Heating Energy",
        "Zone Ideal Loads Supply Air Total Cooling Energy",
        "Zone Ideal Loads Supply Air Total Heating Rate",
        "Zone Ideal Loads Supply Air Total Cooling Rate"
    ]

    for var in output_vars:
        idf.newidfobject("OUTPUT:VARIABLE",
                         Variable_Name=var,
                         Reporting_Frequency="Hourly")

    idf.save(idf_path)

In [10]:
IDF.setiddname("/Applications/EnergyPlus-25-1-0/Energy+.idd")
generate_idf_from_geojson(json.load(open('test/test.geojson', 'r')), 'test/in.idf')